In [5]:
# Load with dill
import dill
with open('schuheinlagen_orakel.dill', 'rb') as f:
    ki_modell = dill.load(f)



In [6]:
help(ki_modell)

Help on SchuheinlagenOrakel in module __main__ object:

class SchuheinlagenOrakel(builtins.object)
 |  SchuheinlagenOrakel(model)
 |  
 |  Dies ist das Schuheinlagen Orakel. Das Orakel ist ein KI-System,
 |  das die Produktion von personalisierten Schuheinlagen unterstützt.
 |  Die einzige Methode, die das KI-System zur Verfügung stellt, ist die
 |  Prognose .predict(). Mehr Details: help(modell.predict)
 |  
 |  Beispiel:
 |  
 |  import pandas as pd 
 |  
 |  bauteile = pd.DataFrame({
 |      'Zellenform': [1, 1],
 |      'Zellengroesse': [3.0, 3.3],
 |      'Fuellgrad': [0.3, 0.38]
 |  })
 |  maximale_kraft = modell.predict(X)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model)
 |  
 |  predict(self, X)
 |      Prognostiziert die maximale Kraft, die von dem Bauteil aufgenommen werden kann.
 |      
 |      Argument X:
 |          Pandas DataFrame: Beschreibung der Eigenschaften des Bauteils.
 |      
 |          Die Spalten des Pandas DataFrames beinhalten 
 |          * '